In [1]:
import pymysql
import requests

下載分析pm2.5 opendata資料

In [2]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [3]:
resp=requests.get(url,verify=False)
resp

c:\Users\user\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [4]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '9',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '14',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '7',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '4',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '4',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '12',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '',
  'datacreationdate': '2025-08-30 22:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '6',
  'datacreationdate':

In [5]:
len(datas)

1000

In [6]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '9',
 'datacreationdate': '2025-08-30 22:00',
 'itemunit': 'μg/m3'}

建立資料表 8/20 1:00:00

In [ ]:
table_str=""" 
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time(site,datacreationdate)
)
"""
print(table_str)

連結資料庫

conn=pymysql.connect(
host="127.0.0.1",
user="root",
password="",
port=3307,
database="demo"
)

In [2]:
conn=pymysql.connect(
host="mysql-c87907a-arthur201010-4fba.j.aivencloud.com",
user="avnadmin",
password="AVNS_HWzrTchr2PZUlSOko5r",
port=11977,
database="defaultdb"
)
cursor= conn.cursor()
cursor

In [ ]:
cursor.execute(table_str)
conn.commit()

寫入一筆資料

In [ ]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

In [ ]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入 8/20 1:40:00
- 加上 ignore 8/20 2:00:00

In [8]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
#values
cursor.executemany(sqlstr,values)
conn.commit()

In [10]:
conn.close()

## 取得各縣市站點的平均值
df.groupby("county")["pm25"].mean()

In [6]:
cursor.execute("select avg(pm25) from pm25;")
float(cursor.fetchone()[0])

7.9666

In [9]:
sqlstr="""
select county, round(avg(pm25),2) from pm25 group by county;
"""
cursor.execute(sqlstr)
cursor.fetchall()

(('彰化縣', Decimal('9.14')),
 ('新北市', Decimal('6.85')),
 ('雲林縣', Decimal('7.60')),
 ('臺東縣', Decimal('3.07')),
 ('澎湖縣', Decimal('5.57')),
 ('金門縣', Decimal('11.38')),
 ('南投縣', Decimal('8.10')),
 ('高雄市', Decimal('9.79')),
 ('桃園市', Decimal('10.15')),
 ('宜蘭縣', Decimal('4.19')),
 ('臺北市', Decimal('6.49')),
 ('花蓮縣', Decimal('4.07')),
 ('屏東縣', Decimal('8.14')),
 ('臺南市', Decimal('7.07')),
 ('嘉義市', Decimal('5.24')),
 ('嘉義縣', Decimal('6.89')),
 ('臺中市', Decimal('9.88')),
 ('苗栗縣', Decimal('10.16')),
 ('新竹市', Decimal('11.48')),
 ('新竹縣', Decimal('10.36')),
 ('基隆市', Decimal('2.89')),
 ('連江縣', Decimal('6.50')))

In [10]:
import pandas as pd

In [14]:
cursor.execute("select * from pm25")
datas=cursor.fetchall()
datas

((1, '員林', '彰化縣', 14, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (2, '富貴角', '新北市', 4, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (3, '麥寮', '雲林縣', 11, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (4, '關山', '臺東縣', 3, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (5, '馬公', '澎湖縣', 7, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (6, '金門', '金門縣', 6, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (7, '埔里', '南投縣', 10, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (8, '復興', '高雄市', 6, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (9, '永和', '新北市', 3, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (10, '竹山', '南投縣', 12, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (11, '中壢', '桃園市', 8, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (12, '三重', '新北市', 7, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (13, '冬山', '宜蘭縣', 7, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (14, '宜蘭', '宜蘭縣', 3, datetime.datetime(2025, 8, 27, 20, 0), 'μg/m3'),
 (15, '陽明'

In [17]:
len(datas)

1751

In [15]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

,id,site,county,pm25,datetime,unit
0,1,員林,彰化縣,14,2025-08-27 20:00:00,μg/m3
1,2,富貴角,新北市,4,2025-08-27 20:00:00,μg/m3
2,3,麥寮,雲林縣,11,2025-08-27 20:00:00,μg/m3
3,4,關山,臺東縣,3,2025-08-27 20:00:00,μg/m3
4,5,馬公,澎湖縣,7,2025-08-27 20:00:00,μg/m3
...,...,...,...,...,...,...
1746,1747,平鎮,桃園市,11,2025-08-30 10:00:00,μg/m3
1747,1748,觀音,桃園市,8,2025-08-30 10:00:00,μg/m3
1748,1749,大園,桃園市,7,2025-08-30 10:00:00,μg/m3
1749,1750,桃園,桃園市,5,2025-08-30 10:00:00,μg/m3


In [16]:
df.groupby("county").get_group("新北市").drop_duplicates()

,id,site,county,pm25,datetime,unit
1,2,富貴角,新北市,4,2025-08-27 20:00:00,μg/m3
8,9,永和,新北市,3,2025-08-27 20:00:00,μg/m3
11,12,三重,新北市,7,2025-08-27 20:00:00,μg/m3
62,63,淡水,新北市,7,2025-08-27 20:00:00,μg/m3
63,64,林口,新北市,5,2025-08-27 20:00:00,μg/m3
...,...,...,...,...,...,...
1694,1695,土城,新北市,6,2025-08-30 11:00:00,μg/m3
1695,1696,新店,新北市,7,2025-08-30 11:00:00,μg/m3
1696,1697,汐止,新北市,4,2025-08-30 11:00:00,μg/m3
1705,1706,永和,新北市,6,2025-08-30 10:00:00,μg/m3


In [18]:
len(df)

1751

In [19]:
df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-08-27 20:00:00
1,富貴角,2025-08-27 20:00:00
2,麥寮,2025-08-27 20:00:00
3,關山,2025-08-27 20:00:00
4,馬公,2025-08-27 20:00:00
...,...,...
1746,平鎮,2025-08-30 10:00:00
1747,觀音,2025-08-30 10:00:00
1748,大園,2025-08-30 10:00:00
1749,桃園,2025-08-30 10:00:00


## 取得單一縣市的站點數值

In [8]:
county="臺北市"
# s="select max(datacreationdate) from pm25"
# 取得最新資料
sqlstr='''
select site,pm25,datacreationdate from pm25
where county=%s and 
datacreationdate=(select max(datacreationdate) from pm25);
'''
cursor.execute(sqlstr,(county,))
datas=cursor.fetchall()
datas

(('大同', 4, datetime.datetime(2025, 8, 31, 1, 0)),
 ('松山', 6, datetime.datetime(2025, 8, 31, 1, 0)),
 ('古亭', 3, datetime.datetime(2025, 8, 31, 1, 0)),
 ('萬華', 3, datetime.datetime(2025, 8, 31, 1, 0)),
 ('中山', 5, datetime.datetime(2025, 8, 31, 1, 0)),
 ('士林', 9, datetime.datetime(2025, 8, 31, 1, 0)))

In [9]:
conn.close()